# Travel Times

In [99]:
# map stuff
import googlemaps
import folium
from folium.plugins import HeatMap

import pandas as pd
import numpy as np
import json

In [100]:
with open('gmaps_key.json') as f:
    key = json.load(f)['key']

gmaps = googlemaps.Client(key = key)

__Test out API__

In [9]:
matrix_result = gmaps.distance_matrix(origins = "SW9 9RA",
                                      destinations = "W12 7TQ",
                                      mode = "transit")
matrix_result

{'destination_addresses': ["Shepherd's Bush, London W12 7TQ, UK"],
 'origin_addresses': ['Nealden St, London SW9 9RA, UK'],
 'rows': [{'elements': [{'distance': {'text': '13.8 km', 'value': 13828},
     'duration': {'text': '48 mins', 'value': 2898},
     'status': 'OK'}]}],
 'status': 'OK'}

__Functions to extract distance and time from API result:__

In [56]:
def get_distance(matrix_result):
    distance = matrix_result['rows'][0]['elements'][0]['distance']['text']
    return distance

def get_time(matrix_result):
    time = matrix_result['rows'][0]['elements'][0]['duration']['text']
    return time

__Create grid of lat longs and call API:__

In [57]:
lats = np.linspace(51.3762, 51.515419, 20)
longs = np.linspace(-0.2271, -0.0982, 10)

travel_times = pd.DataFrame({'lat': [], 'long': [], 'distance': [], 'time': []})

for lat in lats:
    for long in longs:
        matrix_result = gmaps.distance_matrix(origins = (lat, long),
                                              destinations = (51.5141, -0.2271),
                                              mode = "transit")
        distance = get_distance(matrix_result)
        time = get_time(matrix_result)
        
        travel_time = pd.DataFrame({'lat': [lat], 'long': [long],
                               'distance': [distance], 'time': [time]})        
        travel_times = pd.concat([travel_times, travel_time])

__Convert time to minutes and distance to numeric:__

In [79]:
def time_to_minutes(time):
    time = time.split(' hour ')
    if len(time) > 1:
        hours = int(time[0])
        minutes = int(time[1][0:2])
        return hours * 60 + minutes 
    else:
        minutes = int(time[0][0:2])
        return minutes
    
travel_times['distance'] = travel_times['distance'].apply(lambda x: float(x[0:4]))
travel_times['minutes'] = travel_times['time'].apply(time_to_minutes)

travel_times.head()

__Plot initial heatmap, coloured by travel time in minutes:__

In [102]:
# List comprehension to make out list of lists
heat_data = [[row['lat'], row['long'], row['minutes']] for index, row in travel_times.iterrows()]

m = folium.Map(location = [51.45, -0.15], zoom_start = 12)

# Plot it on the map
HeatMap(heat_data, radius = 9, gradient = {.4: 'blue', .65: 'lime', 1: 'red'}).add_to(m)

# Display the map
m